In [1]:
import numpy as np
import pandas as pd
from scipy import stats
from sklearn.linear_model import Ridge, Lasso, ElasticNet, LinearRegression
import plotly.express as px
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.model_selection import cross_val_score

from sklearn import ensemble
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

import time

from catboost import CatBoostRegressor

import re

from statsmodels.stats.outliers_influence import variance_inflation_factor
from xgboost import XGBRegressor
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.svm import SVR

import pickle

pd.set_option('display.max_columns', 100)

In [3]:
def dummify(df, non_dummies, dummies):
    for dummified in dummies:
        for original in non_dummies:
            if original in dummified:
                orig_name = f'{original}_'
                value = dummified.replace(orig_name, '')
                df[dummified] = df[original].map(lambda x: 1 if x == value else 0)
    df=df.drop(non_dummies,axis=1)
    return df

In [4]:
dummies_linear = [   
    'Neighborhood_Blueste',
    'Neighborhood_BrDale',
    'Neighborhood_BrkSide',
    'Neighborhood_ClearCr',
    'Neighborhood_CollgCr',
    'Neighborhood_Crawfor',
    'Neighborhood_Edwards',
    'Neighborhood_Gilbert', 
    'Neighborhood_Greens', 
    'Neighborhood_GrnHill',
    'Neighborhood_IDOTRR',
    'Neighborhood_Landmrk',
    'Neighborhood_MeadowV',
    'Neighborhood_Mitchel',
    'Neighborhood_NAmes',
    'Neighborhood_NPkVill',
    'Neighborhood_NWAmes',
    'Neighborhood_NoRidge',
    'Neighborhood_NridgHt',
    'Neighborhood_OldTown',
    'Neighborhood_SWISU',
    'Neighborhood_Sawyer',
    'Neighborhood_SawyerW',
    'Neighborhood_Somerst',
    'Neighborhood_StoneBr',
    'Neighborhood_Timber',
    'Neighborhood_Veenker',
    'BldgType_2fmCon',
    'BldgType_Duplex',
    'BldgType_Twnhs',
    'BldgType_TwnhsE',
    'MasVnrType_None',
    'MasVnrType_Stone',
    'BSMT_HighQual_bin_500-1000',
    'BSMT_HighQual_bin_0-500',
    'BSMT_HighQual_bin_1000-1500',
    'BSMT_HighQual_bin_1500+',
    'BSMT_LowQual_bin_0-500',
    'BSMT_LowQual_bin_500-1000',
    'BSMT_LowQual_bin_1000-1500',
    'BSMT_LowQual_bin_1500+'
    ]

non_dummies_linear = ['Neighborhood', 'BldgType', 'MasVnrType', 'BSMT_HighQual_bin', 'BSMT_LowQual_bin']

In [9]:
back_end_linear = front_end.copy()
back_end_linear.drop(columns = ['PID', 'SalePrice'], axis =1, inplace = True)
back_end_linear['GrLivArea_log'] = np.log10(back_end_linear['GrLivArea'])
back_end_linear['LotArea_log'] = np.log10(back_end_linear['LotArea'])
back_end_linear['ExterQualDisc'] = back_end_linear['ExterQual'] - back_end_linear['OverallQual']
back_end_linear['OverallCondDisc'] = back_end_linear['OverallCond'] - back_end_linear['OverallQual']
back_end_linear['KitchenQualDisc'] = back_end_linear['KitchenQual'] - back_end_linear['OverallQual']
back_end_linear = back_end_linear.drop(['ExterQual','OverallCond','KitchenQual'], axis=1)

back_end_linear['BSMT_LowQual_bin'] = pd.cut(back_end_linear['BSMT_LowQual'], [-1, 1, 500, 1000, 1500, 2500], labels = ['No basement', '0-500', '500-1000', '1000-1500', '1500+']) 
back_end_linear['BSMT_HighQual_bin'] = pd.cut(back_end_linear['BSMT_HighQual'], [-1, 1, 500, 1000, 1500, 2500], labels = ['No basement', '0-500', '500-1000', '1000-1500', '1500+']) 
back_end_linear.drop(['BSMT_HighQual', 'BSMT_LowQual', 'GrLivArea', 'LotArea'], axis = 1, inplace = True)

back_end_linear = dummify(back_end_linear, non_dummies_linear, dummies_linear)

back_end_svr = back_end_linear.copy()


In [22]:
back_end_svr = pd.DataFrame(svr_backend_scaler.fit_transform(back_end_svr), columns = back_end_linear.columns)


In [21]:
with open('../Matt/SVR_model.pickle', mode = 'rb') as file:
    svr = pickle.load(file)

/opt/anaconda3/envs/geo_env/lib/python3.9/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator SVR from version 0.24.1 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


In [46]:
svr.get_params()

{'C': 100,
 'cache_size': 200,
 'coef0': 0.0,
 'degree': 3,
 'epsilon': 0.1,
 'gamma': 'scale',
 'kernel': 'linear',
 'max_iter': -1,
 'shrinking': True,
 'tol': 0.001,
 'verbose': False}

In [7]:
svr_backend_scaler = StandardScaler()
svr_price_scaler = StandardScaler()

In [8]:
housing_coords = pd.read_csv('../data/ames_housing_price_data_v6.csv', index_col = 0);
housing_coords.drop("SalePrice.1", axis = 1, inplace = True);

y = housing_coords["SalePrice"]
y_std = svr_price_scaler.fit_transform(np.array(np.log10(y)).reshape(-1,1));
front_end = housing_coords.copy();
front_end.reset_index(inplace = True)

In [45]:
svr.score(back_end_svr, y_std)

0.9387299448505945

### What do I want to do on SVR Gaussian kernel?
- Nominal: dummify 
- Ordinal: keep
- Discrete: keep
- Continuous: keep
- Then, StandardScaler it.

Parameters to be tuned:
- gamma: large=overfit, small=underfit
- C: large=overfit, small=underfit
- epsilon: large=underfit, small=overfit

In [23]:
back_end_svr.head() # BSMT: no bin

,OverallQual,house_age_years,GarageCars,FullBath,HalfBath,BsmtExposure_ord,SaleTypeNew,PorchSF,Fireplaces,Pool,BedroomAbvGr,water_tower,graveyard,police,optician,slipway,bar,cinema,supermarket,hotel,stop,farmyard,christian_catholic,jewish,muslim,garden_centre,christian_lutheran,GrLivArea_log,LotArea_log,ExterQualDisc,OverallCondDisc,KitchenQualDisc,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_Greens,Neighborhood_GrnHill,Neighborhood_IDOTRR,Neighborhood_Landmrk,Neighborhood_MeadowV,Neighborhood_Mitchel,Neighborhood_NAmes,Neighborhood_NPkVill,Neighborhood_NWAmes,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,MasVnrType_None,MasVnrType_Stone,BSMT_HighQual_bin_500-1000,BSMT_HighQual_bin_0-500,BSMT_HighQual_bin_1000-1500,BSMT_HighQual_bin_1500+,BSMT_LowQual_bin_0-500,BSMT_LowQual_bin_500-1000,BSMT_LowQual_bin_1000-1500,BSMT_LowQual_bin_1500+
0,-0.060002,1.150896,0.326001,-1.037448,-0.764821,-0.571684,-0.181148,0.747749,0.599562,-0.060659,-1.071957,1.263384,1.130570,1.510657,1.181153,-0.149969,2.679554,-0.585033,1.595874,-0.536106,-0.298637,-0.618438,3.073566,-0.504451,-0.354444,-0.097248,1.433612,-1.568127,-0.209373,-0.810734,0.256247,-0.900607,-0.063953,-0.10346,-0.203908,-0.120261,-0.323822,-0.193949,-0.259598,-0.244049,-0.057178,0.0,-0.151383,0.0,-0.118506,-0.204989,-0.436568,-0.095091,-0.223702,-0.167502,-0.226698,-0.295373,7.770135,-0.240278,-0.213473,-0.246847,-0.131931,-0.145652,-0.092886,-0.144187,-0.179945,-0.198427,-0.294557,0.793817,-0.277897,-0.558061,-0.396829,-0.30902,-0.125387,-0.714681,1.267343,-0.478256,-0.234535
1,-0.811562,-0.412908,-1.037428,0.820273,-0.764821,0.375336,-0.181148,0.166625,-0.943336,-0.060659,-1.071957,0.153288,1.130570,1.510657,1.181153,-0.149969,2.679554,-0.585033,1.595874,-0.536106,-0.298637,-0.618438,3.073566,-0.504451,2.821321,-0.097248,0.488267,-0.933135,-1.420738,2.534396,0.171624,1.932179,-0.063953,-0.10346,-0.203908,-0.120261,-0.323822,-0.193949,3.852105,-0.244049,-0.057178,0.0,-0.151383,0.0,-0.118506,-0.204989,-0.436568,-0.095091,-0.223702,-0.167502,-0.226698,-0.295373,-0.128698,-0.240278,-0.213473,-0.246847,-0.131931,-0.145652,-0.092886,-0.144187,-0.179945,-0.198427,3.394924,-1.259736,-0.277897,1.791920,-0.396829,-0.30902,-0.125387,1.399225,-0.789053,-0.478256,-0.234535
2,-1.563121,2.446245,-1.037428,-1.037448,-0.764821,-0.571684,-0.181148,1.824257,-0.943336,-0.060659,-1.071957,-0.956808,1.130570,1.510657,1.181153,-0.149969,0.592055,1.709305,-0.531862,-0.536106,-0.298637,-0.321205,-0.325355,-0.504451,-0.354444,-0.097248,1.433612,-0.963050,-0.147208,3.537936,2.456444,0.799065,-0.063953,-0.10346,-0.203908,-0.120261,-0.323822,-0.193949,-0.259598,-0.244049,-0.057178,0.0,-0.151383,0.0,-0.118506,-0.204989,-0.436568,-0.095091,-0.223702,-0.167502,-0.226698,3.385547,-0.128698,-0.240278,-0.213473,-0.246847,-0.131931,-0.145652,-0.092886,-0.144187,-0.179945,-0.198427,-0.294557,0.793817,-0.277897,-0.558061,-0.396829,-0.30902,-0.125387,1.399225,-0.789053,-0.478256,-0.234535
3,1.443116,-0.964592,0.326001,0.820273,1.239059,-0.571684,-0.181148,-0.404973,-0.943336,-0.060659,0.180019,-0.956808,-0.656648,-0.661963,-0.846631,-0.149969,-0.600801,1.709305,-0.531862,-0.536106,-0.298637,-0.618438,-0.325355,1.982352,-0.354444,-0.097248,1.433612,0.509692,-0.087430,-0.476221,-0.759228,-0.617328,-0.063953,-0.10346,-0.203908,-0.120261,-0.323822,-0.193949,-0.259598,-0.244049,-0.057178,0.0,-0.151383,0.0,-0.118506,-0.204989,-0.436568,-0.095091,4.470224,-0.167502,-0.226698,-0.295373,-0.128698,-0.240278,-0.213473,-0.246847,-0.131931,-0.145652,-0.092886,-0.144187,-0.179945,-0.198427,-0.294557,0.793817,-0.277897,1.791920,-0.396829,-0.30902,-0.125387,1.399225,-0.789053,-0.4

In [58]:
back_end_svr.shape
y_std.shape

(2455, 1)

In [35]:
kfold = KFold(n_splits=5, shuffle = True)
params = {
    'C': np.logspace(-0.5,1.5,9),
    'epsilon': np.logspace(-2,-1,5),
    'gamma' : np.logspace(-3.5,-2,7)
          }
svr = SVR(kernel = 'rbf')
svr_tuner = GridSearchCV(svr, params, cv=kfold, return_train_score = True, n_jobs = -1)
%time svr_tuner.fit(back_end_svr, y_std.ravel())

CPU times: user 5.11 s, sys: 346 ms, total: 5.46 s
Wall time: 20min 52s


GridSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=True),
             estimator=SVR(), n_jobs=-1,
             param_grid={'C': array([ 0.31622777,  0.56234133,  1.        ,  1.77827941,  3.16227766,
        5.62341325, 10.        , 17.7827941 , 31.6227766 ]),
                         'epsilon': array([0.01      , 0.01778279, 0.03162278, 0.05623413, 0.1       ]),
                         'gamma': array([0.00031623, 0.00056234, 0.001     , 0.00177828, 0.00316228,
       0.00562341, 0.01      ])},
             return_train_score=True)

In [36]:
svr_tuner.best_score_
# 0.9404, 1.5, -1, -3
# 0.9402, 1.25, -1.25, -3

0.9397069092394789

In [37]:
svr_tuner.best_estimator_.get_params()

{'C': 31.622776601683793,
 'cache_size': 200,
 'coef0': 0.0,
 'degree': 3,
 'epsilon': 0.1,
 'gamma': 0.001,
 'kernel': 'rbf',
 'max_iter': -1,
 'shrinking': True,
 'tol': 0.001,
 'verbose': False}

In [39]:
C, gamma, epsilon = (30, 0.001, 0.1);
goal = 0.9450;
parameters = [];

In [40]:
kfold = KFold(n_splits=5, shuffle = True);
svr = SVR(kernel = 'rbf');

round_best = 0.94
for i in range(50):
    step_size = 20*max(0.0005, goal-round_best);
    params = {
    'C': np.linspace((1-step_size)*C, (1+step_size)*C, 5),
    'epsilon': np.linspace((1-step_size)*epsilon, (1+step_size)*epsilon, 5),
    'gamma' : np.linspace((1-step_size)*gamma, (1+step_size)*gamma, 5)
          };
    svr_tuner = GridSearchCV(svr, params, cv=kfold, return_train_score = True, n_jobs = -1)
    %time svr_tuner.fit(back_end_svr, y_std.ravel())
    
    C = svr_tuner.best_estimator_.get_params()["C"]
    epsilon = svr_tuner.best_estimator_.get_params()["epsilon"]
    gamma = svr_tuner.best_estimator_.get_params()["gamma"]
    round_best = svr_tuner.best_score_;
    
    print(C, epsilon, gamma, round_best);
    parameters.append([C, epsilon, gamma, round_best]);


CPU times: user 2.46 s, sys: 231 ms, total: 2.69 s
Wall time: 6min 55s
26.999999999999996 0.11000000000000001 0.0009 0.9401918258378048
CPU times: user 2.41 s, sys: 135 ms, total: 2.55 s
Wall time: 6min 8s
28.298207023792685 0.12057798315682934 0.0008134528650804874 0.9402523613346588
CPU times: user 2.45 s, sys: 132 ms, total: 2.58 s
Wall time: 5min 37s
30.985200260312396 0.11485337620658721 0.0008520726678273736 0.9405872174977026
CPU times: user 2.46 s, sys: 134 ms, total: 2.6 s
Wall time: 5min 52s
33.71981925109013 0.11992160589512926 0.0008520726678273736 0.9411654041559835
CPU times: user 2.47 s, sys: 134 ms, total: 2.6 s
Wall time: 5min 43s
36.30585682631446 0.1245201148108617 0.0008520726678273736 0.9419986667469781
CPU times: user 2.5 s, sys: 135 ms, total: 2.63 s
Wall time: 5min 40s
34.12653731886908 0.11704558758582288 0.0008009255871885464 0.9401651195393995
CPU times: user 2.43 s, sys: 131 ms, total: 2.56 s
Wall time: 5min 47s
35.7765146035786 0.1170455875858229 0.00072347

In [41]:
svr_tuner.cv_results_["mean_test_score"].mean()

0.940574698739808

In [42]:
parameters2 = [];

In [43]:
kfold = KFold(n_splits=5, shuffle = True);
svr = SVR(kernel = 'rbf', epsilon = 0.1);

round_best = 0.94
for i in range(50):
    step_size = 30*max(0.0005, goal-round_best);
    params = {
    'C': np.linspace((1-step_size)*C, (1+step_size)*C, 7),
    # 'epsilon': np.linspace((1-step_size)*epsilon, (1+step_size)*epsilon, 5),
    'gamma' : np.linspace((1-step_size)*gamma, (1+step_size)*gamma, 7)
          };
    svr_tuner = GridSearchCV(svr, params, cv=kfold, return_train_score = True, n_jobs = -1)
    %time svr_tuner.fit(back_end_svr, y_std.ravel())
    
    C = svr_tuner.best_estimator_.get_params()["C"]
    gamma = svr_tuner.best_estimator_.get_params()["gamma"]
    round_best = svr_tuner.best_score_;
    round_mean = svr_tuner.cv_results_["mean_test_score"].mean();
    
    print(C, gamma, round_best, round_mean);
    parameters2.append([C, gamma, round_best, round_mean]);


CPU times: user 1.65 s, sys: 155 ms, total: 1.8 s
Wall time: 2min 56s
217.50920186805013 0.00027951242874910625 0.9393675562575075 0.9391712522847822
CPU times: user 2.52 s, sys: 62.5 ms, total: 2.58 s
Wall time: 2min 59s
242.01136872797485 0.00027951242874910625 0.9411791059397787 0.9410554733988536
CPU times: user 2.61 s, sys: 61.6 ms, total: 2.67 s
Wall time: 3min 1s
260.5053647535495 0.00027951242874910625 0.9395821217308494 0.9394897984958064
CPU times: user 2.7 s, sys: 60.4 ms, total: 2.76 s
Wall time: 3min 3s
274.6192283005033 0.00027951242874910625 0.9413300000654772 0.9411618715036524
CPU times: user 2.69 s, sys: 53.5 ms, total: 2.74 s
Wall time: 3min 4s
304.85480479694957 0.00025899621684495584 0.9405146589671387 0.9404343900687406
CPU times: user 1.68 s, sys: 53.3 ms, total: 1.73 s
Wall time: 3min 5s
318.52858244715617 0.00024737935325725075 0.9415591974029567 0.941463083830464
CPU times: user 2.75 s, sys: 53.3 ms, total: 2.81 s
Wall time: 3min 7s
340.4484619214899 0.0002473

In [44]:
temp = parameters2
temp.sort(key = lambda x: x[3], reverse = True)

In [47]:
np.mean([x[3] for x in temp])

0.9405598093700932